In [1]:
# import libraries
#import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import gzip
import pickle
import os
from urllib.request import urlretrieve
import numpy as np
from matplotlib import image


def conv(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_depth = f.shape[1] #f1 = 1 , f2 = 8
    f_size = f.shape[2] 
        

    h_range = int((image.shape[1] - f_size) / s) + 1     # (W - F + 2P) / S  
    w_range = int((image.shape[2] - f_size) / s) + 1     # (W - F + 2P) / S  
    np_o = np.zeros((f_num, h_range, w_range))
    for z in range(f_num): # Number of filters
        for _h in range(h_range):      
            for _w in range(w_range):
                np_o[z, _h, _w] = np.sum(np.multiply(image[:, _h*s : _h*s + f_size, _w*s : _w*s + f_size] , f[z, :, :, :])) + b[z]
    
    return np_o
def conv_alg(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_size = f.shape[2] 
    
    np_o_t = np.zeros((f_num, image.shape[1] + (f_size-1), image.shape[2]+ (f_size-1)))
    for z in range(f_num): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                np_o_t[z,_h:(_h+f_size),_w:(_w+f_size)] += np.sum((image[:, _h, _w].reshape(-1,1,1)*f[z, :, :, :]),axis=0)
        np_o_t[z,:,:] += b[z]*np.ones((image.shape[1] + (f_size-1),image.shape[2] + (f_size-1)))
    return np_o_t[:,(f_size-1):-(f_size-1), (f_size-1):-(f_size-1)]

def convTransp(image, params, s = 1, pad = 1):
    #s is always 1, upsample kernel = 2
    #zero insertions between pixels s_downsampled -1 = 2-1 = 1
    #required padding in order to double my dimensions with the given data:
    #(i-1)*2 + k -2p = output size, where our padding is k - p -1 = 2-0-1=1(we assume p=0)
    [f, b]=params
    f = np.rot90(f, 2, (2,3))
    params = [f, b]
    input_s = image.shape[1]
    target_dim = input_s*2 #final dim, after conv
    required_pad = 1 #always for filter 2x2 ,stride 1, zero insertion 1 and main target to double dim
    #make our new custom input
    size = input_s*2 +1
    new_in = np.zeros((image.shape[0], size, size))
    
    ### OR just: np.pad(image[:,:,:],2,'constant') # Important, we must loop with respect to the 1st dim
    for i in range(1, target_dim, 2):
        for j in range(1, target_dim, 2):
                new_in[:, i, j] = image[:, i//2, j//2]
    #now we do a normal convolution(pad = 0)
    res = conv(new_in, params, 1, 0) #thats the final result with target_dim
    return res, new_in # we will need new_in so we wont calc it again for the backprop
def convTransp_alg(image, params, s = 2, pad = 1):
    #s is always 1, upsample kernel = 2
    #zero insertions between pixels s_downsampled -1 = 2-1 = 1
    #required padding in order to double my dimensions with the given data:
    #(i-1)*2 + k -2p = output size, where our padding is k - p -1 = 2-0-1=1(we assume p=0)
    [f, b]=params
    f_size = f.shape[2]
    res = np.zeros((f.shape[0], 2*image.shape[1], 2*image.shape[2]))
    for z in range(f.shape[0]): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                res[z, _h*s : _h*s + f_size, _w*s : _w*s + f_size] += np.sum((image[:, _h, _w].reshape(-1,1,1)) * f[z], axis=0)
        res[z,:,:] += b[z]*np.ones((image.shape[1]*2,image.shape[2]*2))
    return res # we will need new_in so we wont calc it again for the backprop

In [2]:
x = np.ones((4,4,4))

for c in range(4):
    for i in range(4):
        for j in range(4):
            x[c,i,j] = (i+1)*(j*2+1)*1.3 + c
print(x)
#k = np.random.rand(1,2,3,3)
k=np.ones((2,4,2,2))

counter = 1
for n in range(2):
    for c in range(4):
        for i in range(2):
            for j in range(2):
                k[n,c,i,j] = counter
                counter=counter+1

print(k)


b=np.zeros((2))

[[[ 1.3  3.9  6.5  9.1]
  [ 2.6  7.8 13.  18.2]
  [ 3.9 11.7 19.5 27.3]
  [ 5.2 15.6 26.  36.4]]

 [[ 2.3  4.9  7.5 10.1]
  [ 3.6  8.8 14.  19.2]
  [ 4.9 12.7 20.5 28.3]
  [ 6.2 16.6 27.  37.4]]

 [[ 3.3  5.9  8.5 11.1]
  [ 4.6  9.8 15.  20.2]
  [ 5.9 13.7 21.5 29.3]
  [ 7.2 17.6 28.  38.4]]

 [[ 4.3  6.9  9.5 12.1]
  [ 5.6 10.8 16.  21.2]
  [ 6.9 14.7 22.5 30.3]
  [ 8.2 18.6 29.  39.4]]]
[[[[ 1.  2.]
   [ 3.  4.]]

  [[ 5.  6.]
   [ 7.  8.]]

  [[ 9. 10.]
   [11. 12.]]

  [[13. 14.]
   [15. 16.]]]


 [[[17. 18.]
   [19. 20.]]

  [[21. 22.]
   [23. 24.]]

  [[25. 26.]
   [27. 28.]]

  [[29. 30.]
   [31. 32.]]]]


In [5]:
params = [k,b]
res_ref,_ = convTransp(x,params)#padding = 0
res = convTransp_alg(x,params)
#print(res_ref)
#res = conv(x,params,1,1)
print(res)

[[[  98.4  109.6  171.2  192.8  244.   276.   316.8  359.2]
  [ 120.8  132.   214.4  236.   308.   340.   401.6  444. ]
  [ 134.8  151.2  280.4  317.6  426.   484.   571.6  650.4]
  [ 167.6  184.   354.8  392.   542.   600.   729.2  808. ]
  [ 171.2  192.8  389.6  442.4  608.   692.   826.4  941.6]
  [ 214.4  236.   495.2  548.   776.   860.  1056.8 1172. ]
  [ 207.6  234.4  498.8  567.2  790.   900.  1081.2 1232.8]
  [ 261.2  288.   635.6  704.  1010.  1120.  1384.4 1536. ]]

 [[ 277.6  288.8  516.8  538.4  756.   788.   995.2 1037.6]
  [ 300.   311.2  560.   581.6  820.   852.  1080.  1122.4]
  [ 397.2  413.6  875.6  912.8 1354.  1412.  1832.4 1911.2]
  [ 430.   446.4  950.   987.2 1470.  1528.  1990.  2068.8]
  [ 516.8  538.4 1234.4 1287.2 1952.  2036.  2669.6 2784.8]
  [ 560.   581.6 1340.  1392.8 2120.  2204.  2900.  3015.2]
  [ 636.4  663.2 1593.2 1661.6 2550.  2660.  3506.8 3658.4]
  [ 690.   716.8 1730.  1798.4 2770.  2880.  3810.  3961.6]]]


In [4]:
print(res[0]-res_ref[0])

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  5.68434189e-14]
 [ 1.42108547e-14  0.00000000e+00 -2.84217094e-14  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.68434189e-14  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.13686838e-13]
 [ 0.00000000e+00  0.00000000e+00 -5.68434189e-14  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.13686838e-13  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.68434189e-14
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.13686838e-13]
 [-2.84217094e-14  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.27373675e-13  0.00000000e+00]
 [ 0.00000000e+00 -2.84217094e-14  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  2.27373675e-13]
 [ 0.00000000e+00  0.00000000e+00 -1.13686838e-13  0.00000000e+00
   